In [3]:
import random
import time
import pygame


class button():
    def __init__(self,color,x,y,width,height,tsize,text=''):
        self.color=color
        self.x=x
        self.y=y
        self.width=width
        self.height=height
        self.text=text
        self.tsize=tsize

    def drawbutton(self,WIN,outline=None):
        if outline:
            pygame.draw.rect(WIN,outline,(self.x-2,self.y-2,self.width+4,self.height+4),0)
        pygame.draw.rect(WIN,self.color,(self.x,self.y,self.width,self.height),0)
        if self.text != '':
            myfont = pygame.font.SysFont('Comic Sans MS', self.tsize)
            text = myfont.render(self.text, 1, (0, 0, 0))
            WIN.blit(text,(self.x+(self.width/2-text.get_width()/2),self.y+(self.height/2-text.get_height()/2)))

    def isOverSquare(self,pos):
        if pos[0]>self.x and pos[0]<self.x+self.width:
            if pos[1]>self.y and pos[1]<self.y+self.height:
                return True
        return False

class slide():
    def __init__(self,row,col):
        self.col=col
        self.row=row
        self.win=False
        self.gameOver=False
        self.board=[]
        self.createBoard()
    

    def createBoard(self):
        count=1
        maxx=self.row*self.col
        for i in range(self.row):
            temp=[]
            for j in range(self.col):
                if count<maxx:
                    temp.append(count)
                else:
                    temp.append(' ')
                count+=1
            self.board.append(temp)
    
    def printBoard(self):
        s=""
        for i in range(self.row):
            for j in range(self.col):
                s+=str(self.board[i][j])
                s+=" "
            s+="\n"
        print(s)

    def findNeighbors(self):
        found=False
        spots=[]
        spot=-1
        for i in range(self.row):
            for j in range(self.col):
                if found==False:
                    if self.board[i][j]==' ':
                        found = True
                        spot=(i,j)
                        if i-1>=0:
                            spots.append([self.board[i-1][j],(i-1,j)])
                        if i+1<self.row:
                            spots.append([self.board[i+1][j],(i+1,j)])
                        if j-1>=0:
                            spots.append([self.board[i][j-1],(i,j-1)])
                        if j+1<self.col:
                            spots.append([self.board[i][j+1],(i,j+1)])
        return spots , spot

    def shuffle(self,buttons):
        maxx=self.row*self.col
        for i in range(maxx**2):
            spots , spot = self.findNeighbors()           
            x=random.randint(0,len(spots)-1)
            rownew=spot[0]
            colnew=spot[1]
            rowold=spots[x][1][0]
            colold=spots[x][1][1]
            self.board[spot[0]][spot[1]]=spots[x][0]
            buttons[spot[0]][spot[1]].text=str(spots[x][0])
            self.board[spots[x][1][0]][spots[x][1][1]]=' '
            buttons[spots[x][1][0]][spots[x][1][1]].text=' '
            tmp=buttons[rowold][colold].color
            buttons[rowold][colold].color=buttons[rownew][colnew].color
            buttons[rownew][colnew].color=tmp

        x=self.checkBoard()
        while(x==True):
            self.shuffle(buttons)
            x=self.checkBoard()
        
    def movePiece(self,i,j,buttons):
        found=False
        if i-1>=0:
            if self.board[i-1][j]==' ':
                self.board[i-1][j]=self.board[i][j]
                self.board[i][j]=' '
                buttons[i-1][j].text=buttons[i][j].text
                buttons[i][j].text=' '
                tmp=buttons[i][j].color
                buttons[i][j].color=buttons[i-1][j].color
                buttons[i-1][j].color=tmp
                found=True
        if i+1<self.row and found==False:
            if self.board[i+1][j]==' ':
                self.board[i+1][j]=self.board[i][j]
                self.board[i][j]=' '
                buttons[i+1][j].text=buttons[i][j].text
                buttons[i][j].text=' '
                tmp=buttons[i][j].color
                buttons[i][j].color=buttons[i+1][j].color
                buttons[i+1][j].color=tmp
                found=True
        if j-1>=0 and found==False:
            if self.board[i][j-1]==' ':
                self.board[i][j-1]=self.board[i][j]
                self.board[i][j]=' '
                buttons[i][j-1].text=buttons[i][j].text
                buttons[i][j].text=' '
                tmp=buttons[i][j].color
                buttons[i][j].color=buttons[i][j-1].color
                buttons[i][j-1].color=tmp
                found=True
        if j+1<self.col and found==False:
            if self.board[i][j+1]==' ':
                self.board[i][j+1]=self.board[i][j]
                self.board[i][j]=' '
                buttons[i][j+1].text=buttons[i][j].text
                buttons[i][j].text=' '
                tmp=buttons[i][j].color
                buttons[i][j].color=buttons[i][j+1].color
                buttons[i][j+1].color=tmp
                found=True
              
    def checkBoard(self):
        count=1
        check=0
        maxx=self.row*self.col
        for i in range(self.row):
            for j in range(self.col):
                if count<maxx:
                    if self.board[i][j]==count:
                        check+=1
                count+=1
        if check==maxx-1:
            self.gameOver=True
      

pygame.init()
pygame.font.init()
WIDTH,HEIGHT =500,700  
WIN=pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("SWIPE PUZZLE")

PINK=(255,0,127)
RED=(255,0,0)
ORANGE=(255,128,0)
YELLOW=(255,255,0)
GREEN=(0,205,0)
CYAN=(0,255,255)
BLUE=(0,0,255)
PURPLE=(102,0,204)
WHITE=(255,255,255)
BLACK=(0,0,0)
LIGHTGRAY=(200,200,200)
LIGHTBLUE=(150,200,255)



def createButtons(w,h,game):
    buttons=[]
    x1=5
    x2=5
    y=100

    start=[(0,205,255,10,-10,-10),(205,155,255,-10,-10,-10),(255,155,200,-10,-10,-10),(0,155,0,10,-10,10),(75,0,155,10,10,10)]
    x=random.randint(0,4)
    r=start[x][0]
    g=start[x][1]
    b=start[x][2]
    cr=start[x][3]
    cg=start[x][4]
    cb=start[x][5]

    for i in range(game.row):
        bu=[]
        for j in range(game.col):
            bu.append(button((r,g,b),x2,y,w-10,h-10,30,str(game.board[i][j])))
            x2+=w
            r=(r+cr)
            if r>=255:
                r-=10
                cr=-1*cr
            if r<=0:
                r+=10
                cr=-1*cr
            g=(g+cg)
            if g>=255:
                g-=10
                cg=-1*cg
            if g<=0:
                g+=10
                cg=-1*cg
            b=(b+cb)
            if b>=255:
                b-=10
                cb=-1*cb
            if b<=0:
                b+=10
                cb=-1*cb
            
        buttons.append(bu)
        x2=x1
        y+=h


    menu=button(WHITE,200,25,100,50,30,"MENU")
    return buttons, menu


def redrawWindow(win,game,buttons,st,bestTime,menu):
    win.fill(LIGHTBLUE)
    txt ="Time:%.2f" % (time.time() - st)
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    text=myfont.render(txt,1,BLACK )
    WIN.blit(text,(400 +(10-text.get_width()/2),50+(15-text.get_height()/2)))

    for i in range(len(buttons)):
        for j in range(len(buttons[i])):
            buttons[i][j].drawbutton(win)

    if bestTime!=None:
        txt ="best Time:%.2f" % bestTime
    else:
        txt="best Time:"
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    text=myfont.render(txt,1,BLACK )
    WIN.blit(text,(400 +(10-text.get_width()/2),600+(15-text.get_height()/2)))

    menu.drawbutton(win)

    pygame.display.update()
    

def gameOver(win,game,buttons,end,bestTime,menu):
    win.fill(LIGHTBLUE)
    txt ="Time:%.2f" % (end)
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    text=myfont.render(txt,1,BLACK )
    WIN.blit(text,(400 +(10-text.get_width()/2),50+(15-text.get_height()/2)))

    for i in range(len(buttons)):
        for j in range(len(buttons[i])):
            buttons[i][j].drawbutton(win)
   
    if bestTime!=None:
        txt ="best Time:%.2f" % bestTime
    else:
        txt="best Time:"
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    text=myfont.render(txt,1,BLACK )
    WIN.blit(text,(400 +(10-text.get_width()/2),600+(15-text.get_height()/2)))

    pygame.draw.rect(win,WHITE,(75,200,350,300))
    myfont = pygame.font.SysFont('Comic Sans MS', 50)
    text=myfont.render("game over",1,BLACK )
    WIN.blit(text,(200 +(50-text.get_width()/2),200+(60-text.get_height()/2)))
    myfont = pygame.font.SysFont('Comic Sans MS', 25)
    text=myfont.render("Do you want to play again?",1,BLACK )
    WIN.blit(text,(200 +(50-text.get_width()/2),300+(25-text.get_height()/2)))
    yes=button(GREEN,150,400,70,50,30,"YES")
    no=button(RED,300,400,70,50,30,"NO")
    yes.drawbutton(win)
    no.drawbutton(win)
    menu.drawbutton(win)
    pygame.display.update()
    play=True
    while play:
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                play=False
                pygame.quit() 
                return False
            if event.type==pygame.MOUSEBUTTONDOWN:
                pos=pygame.mouse.get_pos()
                if yes.isOverSquare(pos):
                    game.gameOver=False
                    play=False
                elif no.isOverSquare(pos):
                    play=False
                elif menu.isOverSquare(pos):
                    play=False
                    return "restart"
            
def pickSize(WIN):
    WIN.fill(LIGHTBLUE)
    levels=[]
    exp=["3x3","4x4","5x5","6x6","3x4","4x5","5x6","6x7"]
    easy1=button(YELLOW,20,75,210,75,20,"SMALL SQUARE")
    medium1=button(GREEN,20,225,210,75,20,"MEDIUM SQUARE")
    hard1=button(CYAN,20,375,210,75,20,"LARGE SQUARE")
    xhard1=button(BLUE,20,525,210,75,15,"EXTRA LAREGE SQUARE")
    easy2=button(ORANGE,260,75,210,75,20,"SMALL RECTANGLE")
    medium2=button(RED,260,225,210,75,18,"MEDIUM RECTANGLE")
    hard2=button(PINK,260,375,210,75,20,"LARGE RECTANGLE")
    xhard2=button(PURPLE,260,525,210,75,14,"EXTRA LAREGE RECTANGLE")
    levels.append(easy1)
    levels.append(medium1)
    levels.append(hard1)
    levels.append(xhard1)
    levels.append(easy2)
    levels.append(medium2)
    levels.append(hard2)
    levels.append(xhard2)

    for i in range(len(levels)):
        pygame.draw.rect(WIN,levels[i].color,(levels[i].x,levels[i].y,levels[i].width,levels[i].height),0)
        myfont = pygame.font.SysFont('Comic Sans MS', levels[i].tsize)
        text=myfont.render(levels[i].text,1,BLACK )
        WIN.blit(text,(levels[i].x +(levels[i].width/2-text.get_width()/2),levels[i].y+(levels[i].height/3-text.get_height()/3)))
        myfont = pygame.font.SysFont('Comic Sans MS', 12)
        text = myfont.render(exp[i],1,BLACK )
        WIN.blit(text,(levels[i].x +(levels[i].width/2-text.get_width()/2),levels[i].y+((levels[i].height/3)*2-(text.get_height()/3))))
    
    pygame.display.update()
    return levels


def main():
    menu=True
    while menu:
        run=True
        size=True
        r=None
        c=None
        while run==True and size==True:
            levels=pickSize(WIN)
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    run=False
                    size=False
                    menu=False
                    pygame.quit()
                if event.type ==pygame.MOUSEBUTTONDOWN:
                    pos=pygame.mouse.get_pos()
                    if levels[0].isOverSquare(pos):
                        r=3
                        c=3
                        size=False
                    elif levels[1].isOverSquare(pos):
                        r=4
                        c=4
                        size=False
                    elif levels[2].isOverSquare(pos):
                        r=5
                        c=5
                        size=False
                    elif levels[3].isOverSquare(pos):
                        r=6
                        c=6
                        size=False
                    elif levels[4].isOverSquare(pos):
                        r=3
                        c=4
                        size=False
                    elif levels[5].isOverSquare(pos):
                        r=4
                        c=5
                        size=False
                    elif levels[6].isOverSquare(pos):
                        r=5
                        c=6
                        size=False
                    elif levels[7].isOverSquare(pos):
                        r=6
                        c=7
                        size=False

        if run==True:
            st = time.time()
            game=slide(r,c)
            end=0
            bestTime=None
            w=int(WIDTH/game.col)
            h=int((HEIGHT-200)/game.row)
            buttons, menu=createButtons(w,h,game)
            game.shuffle(buttons)
   
        while(run):
            redrawWindow(WIN,game,buttons,st,bestTime,menu)
            if game.gameOver==True:
                end=time.time() - st
                if bestTime==None:
                    bestTime=end
                else:
                    bestTime=min(bestTime,end)
                res =gameOver(WIN,game,buttons,end,bestTime,menu)
                if res=="restart":
                    run=False
                else:
                    if game.gameOver==False:
                        st = time.time()
                        game=slide(r,c)
                        buttons, menu=createButtons(w,h,game)
                        game.shuffle(buttons)
                        redrawWindow(WIN,game,buttons,st,bestTime,menu)
                    if game.gameOver==True:
                        run=False
                        menu=False
            if run==True:
                for event in pygame.event.get():
                    if event.type==pygame.QUIT:
                        run=False
                        menu=False
                        pygame.quit()           
                    if event.type==pygame.MOUSEBUTTONDOWN:
                        pos=pygame.mouse.get_pos()
                        for i in range(len(buttons)):
                            for j in range(len(buttons[0])):
                                    if buttons[i][j].isOverSquare(pos):
                                        game.movePiece(i,j,buttons)
                                        game.checkBoard()
                        if menu.isOverSquare(pos):
                            run=False

    pygame.quit()
                                



if __name__ =="__main__":
    main()
